In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re
import time
import numpy as np

In [32]:
ano = '202122'
url = 'https://fbref.com/pt/comps/32/2021-2022/cronograma/2021-2022-Primeira-Liga-Resultados-e-Calendarios'
playoff = True

In [33]:
df_update = pd.read_html(url, extract_links="all")
df_update = df_update[0]

df_update.columns = [col[0] for col in df_update.columns]

df_update_getSecondTuple = ['Relatório da Partida']

for column in df_update.columns:
    if column not in df_update_getSecondTuple:
        df_update[column] = df_update[column].apply(lambda x: x[0])
    else:
        df_update[column] = df_update[column].apply(lambda x: x[1])
        
if playoff:
    df_update = df_update[:-2]
    df_update.drop(['Rodada'],inplace=True,axis=1)
            
df_update['goals_home'] = df_update['Resultado'].str.split('–').str[0]
df_update['goals_away'] = df_update['Resultado'].str.split('–').str[1]
df_update.drop(['Resultado','Notas'],axis=1,inplace=True)
values = ['None']
df_update = df_update[df_update['Relatório da Partida'].isna() == False].reset_index(drop=True) 
df_update.rename({'xG': 'xG_home', '(': 'xG_away', 'Em casa': 'home', 'Visitante': 'away', 'Público': 'Assistance', 'Sem': 'Jornada'}, axis=1, inplace=True)

In [11]:
df_update

,Jornada,Dia,Data,Horário,home,xG_home,xG_away,away,Assistance,Local,Árbitro,Relatório da Partida,goals_home,goals_away
0,1,sex,2021-08-06,20:15,Sporting CP,2.4,0.3,Vizela,9.066,Estádio José Alvalade,António Nobre,/pt/partidas/d678e271/Sporting-CP-Vizela-2021A...,3,0
1,1,sáb,2021-08-07,12:45,Arouca,1.0,1.8,Estoril,513,Estádio Municipal de Arouca,Hugo Coelho,/pt/partidas/066f28ef/Arouca-Estoril-2021Agost...,0,2
2,1,sáb,2021-08-07,18:00,Moreirense,1.7,1.4,Benfica,1.498,P.D. Comendador Joaquim de Almeida Freit...,Vitor Ferreira Fernandes,/pt/partidas/8286ef6e/Moreirense-Benfica-2021A...,1,2
3,1,sáb,2021-08-07,20:30,Marítimo,0.2,1.1,Braga,1.782,Estádio dos Barreiros,André Narciso,/pt/partidas/78abf395/Maritimo-Braga-2021Agost...,0,2
4,1,dom,2021-08-08,15:30,Tondela,1.7,1.5,Santa Clara,764,Estádio João Cardoso,João Ferreira,/pt/partidas/5e7ab169/Tondela-Santa-Clara-2021...,3,0
5,1,dom,2021-08-08,15:30,Vitória,2.0,0.9,Portimonense,8.224,Estádio Dom Afonso Henriques,Fábio Oliveira Melo,/pt/partidas/8d4a7b85/Vitoria-Guimaraes-Portim...,0,1
6,1,dom,2021-08-08,18:00,Porto,2.1,0.9,B-SAD,15.429,Estádio Do Dragão,Gustavo Correia,/pt/partidas/ed25ca34/Porto-Belenenses-SAD-202...,2,0
7,1,dom,2021-08-08,20:30,Paços,3.2,1.1,Famalicão,1.743,Estádio da Capital do Móvel,Miguel Nogueira,/pt/partidas/60adef62/Pacos-de-Ferreira-Famali...,2,0
8,1,seg,2021-08-09,20:15,Gil Vicente FC,1.2,0.8,Boavista,2.014,Estádio Cidade de Barcelos,Cláudio Ruivo Pereira,/pt/partidas/d0566319/Gil-Vicente-FC-Boavista-...,3,0
9,2,sex,2021-08-13,20:15,Estoril,1.1,0.8,Vitória,876,Estádio António Coimbra da Mota,João Ferreira,/pt/partidas/450ff19d/Estoril-Vitoria-Guimarae...,0,0


In [27]:
pd.read_html(str(table))[0]

,B-SAD (4-4-2),B-SAD (4-4-2).1
0,31,João Monteiro
1,99,Álvaro Ramalho
2,2,Diogo Calila
3,20,Rafael Santos
4,22,Henrique Ribeiro
5,33,Trova Boni
6,34,André Lopes
7,42,António Montez
8,44,Eduardo Kau
9,Banco,Banco


In [34]:
try:
    df = pd.read_csv(f'fbref/dataset_{ano}.csv')
    #df.drop('Unnamed: 0',inplace=True)
except:
    df = df_update.copy()
    
df_update['Assistance'] =  df_update['Assistance'].str.replace('.', '').replace('', 0)
df_update['Assistance'] = df_update['Assistance'].astype(int)
df['Assistance'] = df_update['Assistance']

for index, row in df[df['Relatório da Partida'].str.startswith('/pt/stathead/')].iterrows():
    df.loc[index] = df_update.loc[index]

for index, row in df.iterrows():
    
    if row['Relatório da Partida'].startswith('/pt/partidas/') and ('jogador1_home' not in df.columns or pd.isna(df.loc[index,'jogador1_home'])):
    
        start = time.time()

        url = row['Relatório da Partida']
        print(url)
        r = requests.get('http://fbref.com' + url)
        soup = BeautifulSoup(r.content, 'html.parser')
        tables = soup.find_all('table')
        try:
            table = tables[0]
            table2 = tables[1] 
        except Exception:
            indice = index
        try:
            home_players = pd.read_html(str(table))[0].drop(11)
            away_players = pd.read_html(str(table2))[0].drop(11)
        except:
            continue
        df.loc[index,'formacao_home'] = home_players.columns[0].split('(')[1][:-1]
        df.loc[index,'formacao_away'] = away_players.columns[0].split('(')[1][:-1]
        home_players = home_players.transpose().reset_index().drop('index',axis=1)
        away_players = away_players.transpose().reset_index().drop('index',axis=1)

        print(f'meio: {time.time() - start}')
        
        stats_div = soup.find('div', {'id': 'team_stats_extra'})

        # extrai o conteúdo da div
        stats_text = stats_div.text

        linhas = stats_text.split('\n')
        palavras_chave = ['Faltas', 'Escanteios', 'Cruzamentos', 'Contatos', 'Bote defensivo', 'Cortes', 'Jogadas aéreas', 'Defesas', 'Impedimentos', 'Tiro de meta', 'Cobrança de lateral', 'Bolas longas']

        lista_strings = [string for string in linhas for palavra in palavras_chave if palavra in string]

        nova_lista = []
        for string in lista_strings:
            match = re.match(r'^(\d+)([a-zA-Z\s]+)(\d+)$', string)
            if match:
                nova_lista.append([int(match.group(1)), match.group(2).strip(), int(match.group(3))])
        
        for x in nova_lista:
            df.loc[index, f"{x[1]}_home"] = x[0]
            df.loc[index, f"{x[1]}_away"] = x[2]
            
            
        cards = {
            'home_team': {
                'yellow_card': 0,
                'red_card': 0,
                'yellow_red_card': 0
            },
            'away_team': {
                'yellow_card': 0,
                'red_card': 0,
                'yellow_red_card': 0
            }
        }

        home_team_cards_div = soup.find_all('div', class_='cards')[0]
        home_team_cards_spans = home_team_cards_div.find_all('span')
        for span in home_team_cards_spans:
            cards['home_team'][span['class'][0]] += 1

        away_team_cards_div = soup.find_all('div', class_='cards')[1]
        away_team_cards_spans = away_team_cards_div.find_all('span')
        for span in away_team_cards_spans:
            cards['away_team'][span['class'][0]] += 1
        
        df.loc[index, 'yellow_card_home'] = cards['home_team']['yellow_card']
        df.loc[index, 'red_card_home'] = cards['home_team']['red_card']
        df.loc[index, 'yellow_red_card_home'] = cards['home_team']['yellow_red_card']
        df.loc[index, 'yellow_card_away'] = cards['away_team']['yellow_card']
        df.loc[index, 'red_card_away'] = cards['away_team']['red_card']
        df.loc[index, 'yellow_red_card_away'] = cards['away_team']['yellow_red_card']
        
        for i in range(len(home_players.columns) - 1):
            column_name = f'jogador{i+1}_home'
            df.loc[index, column_name] = home_players.iloc[1, i]
        for i in range(len(away_players.columns) - 1):
            column_name = f'jogador{i+1}_away'
            df.loc[index, column_name] = away_players.iloc[1, i]

        print(f'fim: {time.time() - start}')
        if time.time() - start < 6.0:
            time.sleep(6.1 - (time.time() - start))
            
df.to_csv(f'dataset_{ano}.csv')
df.head()

C:\Users\joaof\AppData\Local\Temp\ipykernel_14736\2446060353.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_update['Assistance'] =  df_update['Assistance'].str.replace('.', '').replace('', 0)


/pt/partidas/3606bc7a/Belenenses-SAD-Benfica-2021Novembro27-Primeira-Liga
/pt/partidas/b64f1d37/Maritimo-Pacos-de-Ferreira-2021Novembro28-Primeira-Liga
meio: 1.9866247177124023
fim: 2.088630199432373
/pt/partidas/d290f4ab/Sporting-CP-Tondela-2021Novembro28-Primeira-Liga
meio: 3.59961199760437
fim: 3.703678607940674
/pt/partidas/3d0dca9a/Porto-Vitoria-Guimaraes-2021Novembro28-Primeira-Liga
meio: 3.0861759185791016
fim: 3.1481740474700928
/pt/partidas/b42c9901/Estoril-Santa-Clara-2021Novembro29-Primeira-Liga
meio: 2.538890838623047
fim: 2.686901807785034
/pt/partidas/b98f64f7/Braga-Vizela-2021Novembro30-Primeira-Liga
meio: 4.408685684204102
fim: 4.536675214767456
/pt/partidas/a7f347d4/Portimonense-Porto-2021Dezembro3-Primeira-Liga
meio: 2.1574928760528564
fim: 2.2464840412139893
/pt/partidas/06cbcbf0/Derby-de-Lisboa-Benfica-Sporting-CP-2021Dezembro3-Primeira-Liga
meio: 3.3636832237243652
fim: 3.489694118499756
/pt/partidas/6cd3bd9c/Boavista-Maritimo-2021Dezembro4-Primeira-Liga
meio: 2.40

/pt/partidas/1a2ec4b3/Portimonense-Tondela-2022Janeiro30-Primeira-Liga
meio: 4.047492980957031
fim: 4.205484628677368
/pt/partidas/ff29426e/Vizela-Vitoria-Guimaraes-2022Janeiro30-Primeira-Liga
meio: 2.763920545578003
fim: 2.9419219493865967
/pt/partidas/5f0e146e/Braga-Moreirense-2022Janeiro30-Primeira-Liga
meio: 4.843753099441528
fim: 5.067941188812256
/pt/partidas/f4bb9e0b/Porto-Maritimo-2022Janeiro30-Primeira-Liga
meio: 2.666585683822632
fim: 2.877573013305664
/pt/partidas/1179e0d2/Estoril-Pacos-de-Ferreira-2022Janeiro31-Primeira-Liga
meio: 3.182307243347168
fim: 3.352299928665161
/pt/partidas/843372a9/Famalicao-Arouca-2022Janeiro31-Primeira-Liga
meio: 2.7391717433929443
fim: 2.908968687057495
/pt/partidas/0124e68d/Santa-Clara-Boavista-2022Fevereiro1-Primeira-Liga
meio: 2.5780773162841797
fim: 2.700096607208252
/pt/partidas/414fa528/Benfica-Gil-Vicente-FC-2022Fevereiro2-Primeira-Liga
meio: 4.423205614089966
fim: 4.580209732055664
/pt/partidas/fe84b7c8/Belenenses-SAD-Sporting-CP-2022F

meio: 2.4626622200012207
fim: 2.663726568222046
/pt/partidas/71b4143a/Benfica-Estoril-2022Marco20-Primeira-Liga
meio: 3.3267712593078613
fim: 3.5297679901123047
/pt/partidas/6687ddfa/Portimonense-Braga-2022Marco20-Primeira-Liga
meio: 2.7285337448120117
fim: 2.9015355110168457
/pt/partidas/cd3baef7/Boavista-Porto-2022Marco20-Primeira-Liga
meio: 2.2481091022491455
fim: 2.3681135177612305
/pt/partidas/6e663cb1/Braga-Benfica-2022Abril1-Primeira-Liga
meio: 2.939788818359375
fim: 3.0607922077178955
/pt/partidas/0cd36bea/Belenenses-SAD-Portimonense-2022Abril2-Primeira-Liga
meio: 1.9439141750335693
fim: 2.078916549682617
/pt/partidas/a019f484/Estoril-Vizela-2022Abril2-Primeira-Liga
meio: 2.4392249584198
fim: 2.5932352542877197
/pt/partidas/bc1ebd94/Arouca-Gil-Vicente-FC-2022Abril2-Primeira-Liga
meio: 3.116748571395874
fim: 3.3762874603271484
/pt/partidas/286ff2f7/Famalicao-Boavista-2022Abril2-Primeira-Liga
meio: 2.9127988815307617
fim: 3.0298032760620117
/pt/partidas/39dd2f51/Maritimo-Tondela-

,Unnamed: 0.1,Unnamed: 0,Rodada,Jornada,Dia,Data,Horário,home,xG_home,xG_away,away,Assistance,Local,Árbitro,Relatório da Partida,goals_home,goals_away,formacao_home,formacao_away,Faltas_home,Faltas_away,Escanteios_home,Escanteios_away,Cruzamentos_home,Cruzamentos_away,Contatos_home,Contatos_away,Bote defensivo_home,Bote defensivo_away,Cortes_home,Cortes_away,Defesas_home,Defesas_away,Impedimentos_home,Impedimentos_away,Tiro de meta_home,Tiro de meta_away,Bolas longas_home,Bolas longas_away,yellow_card_home,red_card_home,yellow_red_card_home,yellow_card_away,red_card_away,yellow_red_card_away,jogador1_home,jogador2_home,jogador3_home,jogador4_home,jogador5_home,jogador6_home,jogador7_home,jogador8_home,jogador9_home,jogador10_home,jogador11_home,jogador12_home,jogador13_home,jogador14_home,jogador15_home,jogador16_home,jogador17_home,jogador18_home,jogador19_home,jogador1_away,jogador2_away,jogador3_away,jogador4_away,jogador5_away,jogador6_away,jogador7_away,jogador8_away,jogador9_away,jogador10_away,jogador11_away,jogador12_away,jogador13_away,jogador14_away,jogador15_away,jogador16_away,jogador17_away,jogador18_away,jogador19_away
0,0,0,Regular season,1.0,sex,2021-08-06,20:15,Sporting CP,2.4,0.3,Vizela,9066.0,Estádio José Alvalade,António Nobre,/pt/partidas/d678e271/Sporting-CP-Vizela-2021A...,3,0,3-4-3,4-1-4-1,9.0,16.0,3.0,5.0,26.0,12.0,781.0,460.0,14.0,31.0,12.0,10.0,10.0,19.0,2.0,5.0,5.0,12.0,82.0,45.0,1.0,0.0,0.0,3.0,0.0,0.0,Antonio Adán,Zouhair Feddal,Sebastián Coates,João Palhinha,Matheus Nunes,Jovane Cabral,Rúben Vinagre,Paulinho,Gonçalo Inácio,Pedro Gonçalves,Ricardo Esgaio,Luís Maximiano,Matheus Reis,Bruno Tabata,Nuno Santos,Luís Neto,Tiago Tomás,Gonzalo Plata,Daniel Bragança,Charles,Raphael Guzzo,Cassiano,Kiko Bondoso,Marcos Paulo,Alexis Méndez,Samu,Christian Neiva Afonso,Richard Ofori,Koffi Kouao,Francis Cann,Ivo Gonçalves,Hugo Oliveira,Marcelo,João Pais,Kévin Zohi,Emmanuel Adeyemo,Nuno Gonçalo Moreira,Tomás Costa Silva
1,1,1,Regular season,1.0,sáb,2021-08-07,12:45,Arouca,1.0,1.8,Estoril,513.0,Estádio Municipal de Arouca,Hugo Coelho,/pt/partidas/066f28ef/Arouca-Estoril-2021Agost...,0,2,4-2-3-1,4-3-3,14.0,13.0,3.0,2.0,25.0,13.0,665.0,564.0,12.0,14.0,17.0,11.0,11.0,21.0,4.0,2.0,8.0,9.0,81.0,81.0,1.0,0.0,0.0,3.0,0.0,0.0,Victor Braga,José Manuel Velázquez,André Silva,Arsénio Martins Lafuente Nunes,André Bukia,Pedro Manuel da Silva Moreira,Leandro Miguel Pereira Silva,Eugeni Valderrama,Joel Ferreira,Baptiste Aloé,Thales Oleques,Fernando Castro,Luiz Gustavo,Brunão,Quaresma,Mauro Andrés Caballero,Adílio Santos,Pité,Or Dasa,Daniel Leite,Carles Soria,Bernardo Vital,Lucas Áfrico,Chiquinho,André Clóvis,André Franco,Bruno Lourenço,João Gamboa,Joãozinho,Loreintz Rosier,Mosquito,Francisco Geraldes,Luciano Vega,Racine Coly,Leonardo Ruíz,David Bruno,Ryotaro Meshino,Elias Achouri
2,2,2,Regular season,1.0,sáb,2021-08-07,18:00,Moreirense,1.7,1.4,Benfica,1498.0,P.D. Comendador Joaquim de Almeida Freit...,Vitor Ferreira Fernandes,/pt/partidas/8286ef6e/Moreirense-Benfica-2021A...,1,2,4-4-2,3-4-3,20.0,18.0,0.0,6.0,12.0,8.0,489.0,686.0,22.0,14.0,10.0,19.0,14.0,16.0,1.0,0.0,4.0,5.0,87.0,74.0,4.0,0.0,0.0,4.0,1.0,0.0,Mateus Pasinato,Lazar Rosić,Artur Jorge Marques Amorim,Abdu Conté,Fábio Pacheco,Walterson,Felipe Pires,Yan Santos,Filipe Soares,Abdoulaye Ba,Rafael Martins,Kewin,Ibrahima Camará,André Luis,Sori Mané,Steven Vitória,Godfried Frimpong,Jefferson Galego,Rúben Ramos,Odisseas Vlachodimos,Lucas Veríssimo,Jan Vertonghen,Everton Soares,Luca Waldschmidt,Soualiho Meïté,Diogo Gonçalves,Nicolás Otamendi,Gil Bastião Dias,Adel Taarabt,Gonçalo Ramos,Mile Svilar,Gilberto,Álex Grimaldo,Roman Yaremchuk,Pizzi,Rafa Silva,Julian Weigl,Gedson Fernandes
3,3,3,Regular season,1.0,sáb,2021-08-07,20:30,Marítimo,0.2,1.1,Braga,1782.0,Estádio dos Barreiros,André Narciso,/pt/partidas/78abf395/Maritimo-Braga-2021Agost...,0,2,4-4-2,4-4-2,23.0,12.0,3.0,6.0,13.0,18.0,504.0,604.0,18.0,11.0,15.0,15.0,18.0,7.0,1.0,0.0,10.0,5.0,79.0,97.0,3.0,0.0,0.0,3.0,0.0,0.0,João M

In [ ]:
df.to_csv(f'dataset_{ano}.csv')
df

In [ ]:
### 538
df_538 = pd.read_csv('https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv')
df_538 = df_538[df_538['league'] == 'Portuguese Liga']
df_538 = df_538[df_538['season'] == 2022] 


In [ ]:
df_538['league'].unique()

In [ ]:
df_update = pd.read_html('https://fbref.com/pt/comps/32/cronograma/Primeira-Liga-Resultados-e-Calendarios#sched_2022-2023_32_1', extract_links="all")
df_update = df_update[0]

In [ ]:
import numpy as np
df = pd.read_csv('output_dataset_exemplo.csv')
df.set_index('Unnamed: 0', inplace=True)
df_update['Assistance'] =  df_update['Assistance'].str.replace('.', '').replace('', np.nan)
df_update['Assistance'] = df_update['Assistance'].astype(float)
df['Assistance'] = df_update['Assistance']

In [ ]:
df_update.columns = [col[0] for col in df_update.columns]

df_update_getSecondTuple = ['Relatório da Partida']

for column in df_update.columns:
    if column not in df_update_getSecondTuple:
        df_update[column] = df_update[column].apply(lambda x: x[0])
    else:
        df_update[column] = df_update[column].apply(lambda x: x[1])

In [ ]:
df2 = df_update[df_update['Assistance'].isna() == False]#.reset_index() 
#

In [ ]:
df

In [ ]:
df_diff = df.compare(df_update)

# Seleciona as linhas que são diferentes em df e df_update
row_idx = df_diff.index[df_diff.ne("").any(axis=1)]

# Substitui as linhas em df pelos valores atualizados em df_update
#df.loc[row_idx] = df_update.loc[row_idx]